In [11]:
from gh20_design.acquire import ResolveConfig, resolve_sequences

cfg = ResolveConfig(
    txt_path="../data/GH20.txt",
    db_fasta="../data/CAZyDB.fa",
    out_fasta="../outputs/01_curated.fasta",
    out_missing_tsv="../outputs/01_missing.tsv",
    header_style="rich",
)

report = resolve_sequences(cfg)
print(
    f"Total: {report.total}, "
    f"Matched: {report.matched}, "
    f"Missing: {report.missing}"
)





Total: 29104, Matched: 22028, Missing: 6839


In [6]:
from gh20_design.database import LengthFilterConfig, CDHitConfig, length_filter_fasta, run_cdhit

# Inputs from acquire step
curated = "../outputs/01_curated.fasta"

# Outputs
filtered = "../outputs/02_filtered.fasta"
nr90 = "../outputs/03_nr90.fasta"

# 1) Length filter
lf_cfg = LengthFilterConfig(min_len=200, max_len=None, drop_ambiguous=False)
lf_summary = length_filter_fasta(curated, filtered, lf_cfg)
print("Length filter:", {k: lf_summary[k] for k in ["total", "kept", "dropped_short", "dropped_long", "dropped_ambiguous"]})

# 2) CD-HIT redundancy reduction
cd_cfg = CDHitConfig(identity=0.90, threads=8, memory_mb=16000, description_len=0)
cd_summary = run_cdhit(filtered, nr90, cd_cfg)
print("CD-HIT:", {k: cd_summary[k] for k in ["identity", "word_length", "output", "clusters_file"]})


Length filter: {'total': 22028, 'kept': 21591, 'dropped_short': 437, 'dropped_long': 0, 'dropped_ambiguous': 0}
CD-HIT: {'identity': 0.9, 'word_length': 5, 'output': '../outputs/03_nr90.fasta', 'clusters_file': '../outputs/03_nr90.fasta.clstr'}


In [2]:
from gh20_design.ssn import (
    DiamondConfig, AllVsAllConfig, EdgeFilterConfig,
    make_diamond_db, run_all_vs_all, filter_edges
)

fasta_nr = "../outputs/03_nr90.fasta"

db_prefix = "../outputs/ssn/gh20_nr90"          # creates gh20_nr90.dmnd
hits_tsv  = "../outputs/ssn/04_diamond_hits.tsv"
edges_tsv = "../outputs/ssn/05_network_edges.tsv"

diamond_cfg = DiamondConfig(threads=8, tmpdir=None)

# 1) Build DIAMOND DB
db_summary = make_diamond_db(fasta_nr, db_prefix, diamond_cfg)
print("DIAMOND DB:", db_summary["db_file"])

# 2) All-vs-all search
av_cfg = AllVsAllConfig(
    evalue=1e-5,
    max_target_seqs=5000,     # you can lower (e.g. 500) if runtime is huge
    min_identity=None,
    query_cover=None,
    subject_cover=None,
)
hits_summary = run_all_vs_all(fasta_nr, db_prefix, hits_tsv, diamond_cfg, av_cfg)
print("Hits:", hits_summary["out_tsv"])

# 3) Threshold filtering → edge list
ef_cfg = EdgeFilterConfig(
    bitscore_min=100.0,
    evalue_max=1e-5,
    min_align_len=0,
    min_identity=0.0,
    min_qcov=0.0,
    min_scov=0.0,
    drop_self_hits=True,
)
edge_summary = filter_edges(hits_tsv, edges_tsv, ef_cfg)
print("Edges:", {k: edge_summary[k] for k in ["total_hits", "kept_edges", "edges_tsv"]})


DIAMOND DB: ../outputs/ssn/gh20_nr90.dmnd
Hits: ../outputs/ssn/04_diamond_hits.tsv
Edges: {'total_hits': 19881015, 'kept_edges': 16851098, 'edges_tsv': '../outputs/ssn/05_network_edges.tsv'}


In [2]:
from gh20_design.clustering import (
    EdgeToMCLConfig, MCLConfig,
    edges_tsv_to_abc_stream, run_mcl, clusters_raw_to_tsv
)

edges_tsv = "../outputs/ssn/05_network_edges.tsv"

abc_path = "../outputs/clustering/06_edges.abc"
raw_path = "../outputs/clustering/07_mcl_raw.txt"
tsv_path = "../outputs/clustering/08_clusters.tsv"

# Convert edges to abc (streaming)
abc_cfg = EdgeToMCLConfig(transform="log10", undirected=True, drop_self=True)
abc_summary = edges_tsv_to_abc_stream(edges_tsv, abc_path, abc_cfg)
print("ABC:", abc_summary)

# Run MCL
mcl_cfg = MCLConfig(inflation=2.0, threads=8)
mcl_summary = run_mcl(abc_path, raw_path, mcl_cfg)
print("MCL:", mcl_summary)

# Tidy clusters table
clu_summary = clusters_raw_to_tsv(raw_path, tsv_path, min_size=2)
print("Clusters:", clu_summary)


ABC: {'total_edges_in': 16851098, 'lines_written': 33702196, 'out_abc': '../outputs/clustering/06_edges.abc', 'transform': 'log10'}
MCL: {'out_raw': '../outputs/clustering/07_mcl_raw.txt', 'inflation': 2.0, 'threads': 8}
Clusters: {'total_clusters': 208, 'kept_clusters': 208, 'out_tsv': '../outputs/clustering/08_clusters.tsv', 'min_size': 2}


In [1]:
from gh20_design.consensus import (
    split_fasta_by_cluster, MafftConfig, run_mafft, consensus_from_alignment
)
from pathlib import Path

nr90_fasta = "../outputs/03_nr90.fasta"
clusters_tsv = "../outputs/clustering/08_clusters.tsv"

cluster_fastas_dir = Path("../outputs/consensus/cluster_fastas")
aln_dir = Path("../outputs/consensus/alignments")
cons_dir = Path("../outputs/consensus/consensus_fastas")
final_cons_fasta = Path("../outputs/consensus/09_consensus_all.fasta")

# 1) Split into cluster FASTAs
split_summary = split_fasta_by_cluster(
    fasta_in=nr90_fasta,
    clusters_tsv=clusters_tsv,
    out_dir=cluster_fastas_dir,
    min_cluster_size=2,
)
print("Split:", split_summary)

# 2) MAFFT + consensus per cluster
mafft_cfg = MafftConfig(threads=8, mode="auto")

final_cons_fasta.parent.mkdir(parents=True, exist_ok=True)
with final_cons_fasta.open("w", encoding="utf-8") as out_all:
    for fasta_path in sorted(cluster_fastas_dir.glob("C*.fasta")):
        cid = fasta_path.stem  # e.g. C12
        aln_path = aln_dir / f"{cid}.aln.fasta"
        cons_path = cons_dir / f"{cid}.cons.fasta"

        # Run MAFFT
        run_mafft(fasta_path, aln_path, mafft_cfg)

        # Build consensus
        consensus_from_alignment(
            aln_fasta=aln_path,
            out_fasta=cons_path,
            consensus_id=f"{cid}|consensus",
            min_fraction=0.5,
        )

        # Append to combined FASTA
        out_all.write(cons_path.read_text(encoding="utf-8"))

print("Wrote:", final_cons_fasta)


Split: {'fasta_in': '../outputs/03_nr90.fasta', 'clusters_total': 208, 'clusters_written': 208, 'total_members': 7784, 'missing_members': 0, 'out_dir': '../outputs/consensus/cluster_fastas'}
Wrote: ../outputs/consensus/09_consensus_all.fasta
